<a href="https://colab.research.google.com/github/marciorasf/fuzzy-systems/blob/master/tp1/tp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
from google.colab import files
uploaded = files.upload()

Saving fcm_dataset.mat to fcm_dataset (1).mat
Saving photo002.jpg to photo002 (1).jpg


In [58]:
from scipy.io import loadmat
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from functools import reduce


def euclNorm(arr):
    return np.linalg.norm(arr, ord=2)


def cMeans(data, nClusters, maxIterations=2, mParam=2):
    xDimension = data.shape[1]
    nRows = data.shape[0]
    xColumns = [f"x{i}" for i in range(xDimension)]
    clusterColumns = [f"k{i}" for i in range(nClusters)]

    dfData = pd.DataFrame(data, columns=xColumns)
    dfCentroids = pd.DataFrame(
        np.zeros((nClusters, xDimension)), columns=xColumns, index=clusterColumns,
    )

    for clusterCol in clusterColumns:
        dfData[clusterCol] = 0

    for row_label, _ in dfData.iterrows():
        dfData.loc[row_label, clusterColumns] = np.random.dirichlet(
            np.ones(nClusters), size=1
        )[0]

    for clusterCol in clusterColumns:
        dfCentroids.loc[clusterCol, :] = [
            [np.dot((dfData[clusterCol] ** mParam), dfData[xCol]) for xCol in xColumns]
        ] / (dfData[clusterCol] ** 2).sum()

    exponent = 2 / (mParam - 1)
    for gen in range(maxIterations):
        for clusterCol in clusterColumns:
            X = dfData.loc[:, xColumns]
            denominator = np.zeros((nRows, 1))
            for _, centroid in dfCentroids.iterrows():
                tempNum = np.array(
                    list(
                        map(
                            euclNorm,
                            np.array((X - dfCentroids.loc[clusterCol, xColumns])),
                        )
                    )
                ).reshape((nRows,1))
                tempDen = (
                    np.array(list(map(euclNorm, np.array((X - centroid)))))
                ).reshape((nRows,1))
                tempFrac = (tempNum/tempDen)
                tempRes = tempFrac**exponent
                denominator += tempRes
            dfData.loc[:, clusterCol] = np.ones((nRows, 1)) / denominator

        for clusterCol in clusterColumns:
            dfCentroids.loc[clusterCol, :] = [
                [
                    np.dot((dfData[clusterCol] ** mParam), dfData[xCol])
                    for xCol in xColumns
                ]
            ] / (dfData[clusterCol] ** 2).sum()

        print(f"iteration {gen}")

    return [dfData, dfCentroids]


In [64]:
def printData2d(data, x, y):
    fig = px.scatter(data, x=x, y=y)
    fig.show()

def printCentroids2d(data, x, y):
    fig = px.scatter(centroids, x=x, y=y)
    fig.show()

def printDataAndCentroids2d(data, centroids, keys):
    fig = make_subplots(x_title="x", y_title="y")

    # Add traces
    fig.add_trace(go.Scatter(
        x=data[keys[0]], 
        y=data[keys[1]], 
        mode="markers", 
        name="Input points"
        ))
    fig.add_trace(go.Scatter(
        x=centroids[keys[0]], 
        y=centroids[keys[1]], 
        mode="markers", 
        name="Centroids",
        marker=dict(
            color="red",
            size=15
        )
        ))
    fig.show()

def printDataAndCentroids3d(data, centroids, keys):
    fig = make_subplots(x_title="x", y_title="y")

    # Add traces
    fig.add_trace(go.Scatter3d(
        x=data[keys[0]], 
        y=data[keys[1]], 
        z=data[keys[2]], 
        mode="markers", 
        name="Input points",
        marker=dict(
            color="blue",
            size=2
        )
        ))
    fig.add_trace(go.Scatter3d(
        x=centroids[keys[0]], 
        y=centroids[keys[1]],
        z=centroids[keys[2]], 
        mode="markers", 
        name="Centroids",
        marker=dict(
            color="red",
            size=20
        )
        ))
    fig.show()

In [ ]:
rawData = loadmat(io.BytesIO(uploaded['fcm_dataset.mat']))
part1Data, part1Centroids = cMeans(rawData["x"], 4, 2, 10)

In [ ]:
imageMatrix = imageio.imread(io.BytesIO(uploaded['photo002.jpg']))
flatImage = np.array([item for sublist in imageMatrix for item in sublist])
flatImage = flatImage[0 : : 20]
part2Data, part2Centroids = cMeans(flatImage, 3, 100, 2)
print(part2Centroids)

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16


In [78]:
printDataAndCentroids3d(part2Data, part2Centroids, ["x0", "x1", "x2"])